<h1><center>OpenVaccine || EDA || Feature engineering || Modeling</center></h1>

<center><img src="https://daslab.stanford.edu/site_data/news_img/openvaccine_lores.png"></center>

### In this kernel I am going to present some basic data overview, feature engineering and prepare keras neural network model. Let's fo it and have fun!

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:black; border:0; color:white' role="tab" aria-controls="home"><center>Quick navigation</center></h2>

* [1. Quick Data Overview](#1)
* [2. Sample Analysis](#2)
* [3. Feature Engineering](#3)
* [4. Keras Neural Network Model](#4)
* [5. Prepare submission file](#5)

<a id="1"></a>
<h2 style='background:black; border:0; color:white'><center>1. Quick Data Overview</center><h2>

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import plotly.express as px
from collections import Counter as count

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import KFold

/kaggle/input/stanford-covid-vaccine/train.json
/kaggle/input/stanford-covid-vaccine/test.json
/kaggle/input/stanford-covid-vaccine/sample_submission.csv
/kaggle/input/stanford-covid-vaccine/bpps/id_61a1afa7f.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_c824b643c.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_51af0d845.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_ee4f597e5.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_8956253a1.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_5c2e1a107.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_2f2f6cc40.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_51191ffc3.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_444978315.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_ab723f32f.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_59aafc1ee.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_ca509cada.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_cba8c8d5a.npy
/kaggle/input/stanford-covid-vaccine/bpps/id_32308d54c.npy
/kaggle/input/stanfo

In [2]:
train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)
sub = pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv')

print('Train shapes: ', train.shape)
print('Test shapes: ', test.shape)

Train shapes:  (2400, 19)
Test shapes:  (3634, 7)


### So we have 2400 sequences in training set and 3634 in test set.

In [3]:
train.head(10)

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."
5,5,id_00ab2d761,GGAAAGCGCCGCGGCGGUAGCGGCAGCGAGGAGCGCUACCAAGGCA...,.....(.(((((.(((((((((...........)))))))..(((....,EEEEESISSSSSISSSSSSSSSHHHHHHHHHHHSSSSSSSMMSSSH...,4.136,1,107,68,"[0.1942, 0.2041, 0.1626, 0.1213, 0.10590000000...","[0.2726, 0.2984, 0.2166000

In [4]:
test.head(5)

,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


In [5]:
sub

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.0,0.0,0.0,0.0,0.0
1,id_00073f8be_1,0.0,0.0,0.0,0.0,0.0
2,id_00073f8be_2,0.0,0.0,0.0,0.0,0.0
3,id_00073f8be_3,0.0,0.0,0.0,0.0,0.0
4,id_00073f8be_4,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
457948,id_ffda94f24_125,0.0,0.0,0.0,0.0,0.0
457949,id_ffda94f24_126,0.0,0.0,0.0,0.0,0.0
457950,id_ffda94f24_127,0.0,0.0,0.0,0.0,0.0
457951,id_ffda94f24_128,0.0,0.0,0.0,0.0,0.0


### Let's check all training features

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                2400 non-null   int64  
 1   id                   2400 non-null   object 
 2   sequence             2400 non-null   object 
 3   structure            2400 non-null   object 
 4   predicted_loop_type  2400 non-null   object 
 5   signal_to_noise      2400 non-null   float64
 6   SN_filter            2400 non-null   int64  
 7   seq_length           2400 non-null   int64  
 8   seq_scored           2400 non-null   int64  
 9   reactivity_error     2400 non-null   object 
 10  deg_error_Mg_pH10    2400 non-null   object 
 11  deg_error_pH10       2400 non-null   object 
 12  deg_error_Mg_50C     2400 non-null   object 
 13  deg_error_50C        2400 non-null   object 
 14  reactivity           2400 non-null   object 
 15  deg_Mg_pH10          2400 non-null   o

In [7]:
train.describe()

,index,signal_to_noise,SN_filter,seq_length,seq_scored
count,2400.000000,2400.000000,2400.000000,2400.0,2400.0
mean,1199.500000,4.530456,0.662083,107.0,68.0
std,692.964646,2.835142,0.473099,0.0,0.0
min,0.000000,-0.103000,0.000000,107.0,68.0
25%,599.750000,2.391000,0.000000,107.0,68.0
50%,1199.500000,4.442500,1.000000,107.0,68.0
75%,1799.250000,6.294250,1.000000,107.0,68.0
max,2399.000000,17.194000,1.000000,107.0,68.0


In [8]:
train["seq_scored"].value_counts()

68    2400
Name: seq_scored, dtype: int64

### As we can see every sequence from training set has only 68 scored bases (first 68 bases).

In [9]:
fig = px.histogram(
    train, 
    "signal_to_noise", 
    nbins=25, 
    title='signal_to_noise column distribution', 
    width=700,
    height=500
)
fig.show()

In [10]:
ds = train['SN_filter'].value_counts().reset_index()
ds.columns = ['SN_filter', 'count']
fig = px.pie(
    ds, 
    values='count', 
    names="SN_filter", 
    title='SN_filter bar chart', 
    width=500, 
    height=500
)
fig.show()

In [11]:
train['seq_length'].value_counts()

107    2400
Name: seq_length, dtype: int64

In [12]:
test['seq_length'].value_counts()

130    3005
107     629
Name: seq_length, dtype: int64

In [13]:
train['seq_scored'].value_counts()

68    2400
Name: seq_scored, dtype: int64

In [14]:
test['seq_scored'].value_counts()

91    3005
68     629
Name: seq_scored, dtype: int64

<a id="2"></a>
<h2 style='background:black; border:0; color:white'><center>2. Sample Analysis</center><h2>

### Let's explore 1 sample from train set. We will focus on some columns and see values.

### We have 3 columns that represent structure of sequence: sequence, structure and predicted_loop_type

In [15]:
sample = train.iloc[0]
sample

index                                                                  0
id                                                          id_001f94081
sequence               GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...
structure              .....((((((.......)))).)).((.....((..((((((......
predicted_loop_type    EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...
signal_to_noise                                                    6.894
SN_filter                                                              1
seq_length                                                           107
seq_scored                                                            68
reactivity_error       [0.1359, 0.20700000000000002, 0.1633, 0.1452, ...
deg_error_Mg_pH10      [0.26130000000000003, 0.38420000000000004, 0.1...
deg_error_pH10         [0.2631, 0.28600000000000003, 0.0964, 0.1574, ...
deg_error_Mg_50C       [0.1501, 0.275, 0.0947, 0.18660000000000002, 0...
deg_error_50C          [0.2167, 0.34750000000000003

## 1. Sequence

### We have 4 possible nitrogeneous bases for RNA:

1) Guanine (G) <br>
2) Adenine (A) <br>
3) Cytosine (C) <br>
4) Uracil (U) <br>

#### For more details you can check <a href="https://en.wikipedia.org/wiki/Nucleobase">here.</a>

<center><img src="https://www.thoughtco.com/thmb/jnQVk0_RZ4TRJHeFKR7xxqSV1Pk=/1500x1000/filters:fill(auto,1)/dna-versus-rna-608191_sketch_Final-54acdd8f8af04c73817e8811c32905fa.png" width="700" height="500"></center>

In [16]:
sample['sequence']

'GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC'

In [17]:
dict(count(sample['sequence']))

{'G': 19, 'A': 45, 'C': 23, 'U': 20}

In [18]:
bases = []

for j in range(len(train)):
    counts = dict(count(train.iloc[j]['sequence']))
    bases.append((
        counts['A'] / 107,
        counts['G'] / 107,
        counts['C'] / 107,
        counts['U'] / 107
    ))
    
bases = pd.DataFrame(bases, columns=['A_percent', 'G_percent', 'C_percent', 'U_percent'])
bases

,A_percent,G_percent,C_percent,U_percent
0,0.420561,0.177570,0.214953,0.186916
1,0.233645,0.308411,0.299065,0.158879
2,0.401869,0.224299,0.186916,0.186916
3,0.261682,0.327103,0.327103,0.084112
4,0.542056,0.056075,0.056075,0.345794
...,...,...,...,...
2395,0.439252,0.205607,0.168224,0.186916
2396,0.672897,0.056075,0.196262,0.074766
2397,0.299065,0.252336,0.271028,0.177570
2398,0.448598,0.233645,0.158879,0.158879


### The length of sequence should be equal to value in ```seq_length``` column

In [19]:
len(sample['sequence']) == sample['seq_length']

True

## 2. Structure

In [20]:
sample['structure']

'.....((((((.......)))).)).((.....((..((((((....))))))..)).....))....(((((((....))))))).....................'

### Here we can see 3 different types of characters. Chaacter ```.``` means that base is without pair. ```(``` - is start of pair, ```)``` - the end for current pair. So the number of ```(``` should be equal to ```)```.

In [21]:
dict(count(sample['structure']))

{'.': 61, '(': 23, ')': 23}

In [22]:
pairs_rate = []

for j in range(len(train)):
    res = dict(count(train.iloc[j]['structure']))
    pairs_rate.append(res['('] / 53.5)
    
pairs_rate = pd.DataFrame(pairs_rate, columns=['pairs_rate'])
pairs_rate

,pairs_rate
0,0.429907
1,0.616822
2,0.448598
3,0.598131
4,0.654206
...,...
2395,0.355140
2396,0.130841
2397,0.635514
2398,0.280374


### Let's check all pairs for our sample

In [23]:
pairs_dict = {}
queue = []
for i in range(0, len(sample['structure'])):
    if sample['structure'][i] == '(':
        queue.append(i)
    if sample['structure'][i] == ')':
        first = queue.pop()
        try:
            pairs_dict[(sample['sequence'][first], sample['sequence'][i])] += 1
        except:
            pairs_dict[(sample['sequence'][first], sample['sequence'][i])] = 1
pairs_dict

{('U', 'G'): 2, ('C', 'G'): 3, ('U', 'A'): 6, ('G', 'C'): 8, ('A', 'U'): 4}

In [24]:
pairs = []
all_partners = []
for j in range(len(train)):
    partners = [-1 for i in range(130)]
    pairs_dict = {}
    queue = []
    for i in range(0, len(train.iloc[j]['structure'])):
        if train.iloc[j]['structure'][i] == '(':
            queue.append(i)
        if train.iloc[j]['structure'][i] == ')':
            first = queue.pop()
            try:
                pairs_dict[(train.iloc[j]['sequence'][first], train.iloc[j]['sequence'][i])] += 1
            except:
                pairs_dict[(train.iloc[j]['sequence'][first], train.iloc[j]['sequence'][i])] = 1
                
            partners[first] = i
            partners[i] = first
    
    all_partners.append(partners)
    
    pairs_num = 0
    pairs_unique = [('U', 'G'), ('C', 'G'), ('U', 'A'), ('G', 'C'), ('A', 'U'), ('G', 'U')]
    for item in pairs_dict:
        pairs_num += pairs_dict[item]
    add_tuple = list()
    for item in pairs_unique:
        try:
            add_tuple.append(pairs_dict[item]/pairs_num)
        except:
            add_tuple.append(0)
    pairs.append(add_tuple)
    
pairs = pd.DataFrame(pairs, columns=['U-G', 'C-G', 'U-A', 'G-C', 'A-U', 'G-U'])
pairs

,U-G,C-G,U-A,G-C,A-U,G-U
0,0.086957,0.130435,0.260870,0.347826,0.173913,0.000000
1,0.030303,0.363636,0.030303,0.393939,0.121212,0.060606
2,0.041667,0.208333,0.208333,0.291667,0.125000,0.125000
3,0.000000,0.437500,0.062500,0.406250,0.031250,0.062500
4,0.000000,0.000000,0.457143,0.000000,0.542857,0.000000
...,...,...,...,...,...,...
2395,0.000000,0.210526,0.263158,0.263158,0.157895,0.105263
2396,0.142857,0.000000,0.428571,0.142857,0.285714,0.000000
2397,0.000000,0.294118,0.235294,0.323529,0.147059,0.000000
2398,0.133333,0.200000,0.066667,0.400000,0.200000,0.000000


In [25]:
train['partners'] = all_partners

### Let's do it for all samples

In [26]:
pairs_dict = {}
queue = []
for j in range(len(train)):
    sam = train.iloc[j]
    for i in range(0, len(sam['structure'])):
        if sam['structure'][i] == '(':
            queue.append(i)
        if sam['structure'][i] == ')':
            first = queue.pop()
            try:
                pairs_dict[(sam['sequence'][first], sam['sequence'][i])] += 1
            except:
                pairs_dict[(sam['sequence'][first], sam['sequence'][i])] = 1
                
pairs_dict

{('U', 'G'): 2280,
 ('C', 'G'): 15797,
 ('U', 'A'): 9955,
 ('G', 'C'): 16336,
 ('A', 'U'): 9961,
 ('G', 'U'): 2439}

### Basically I don't know now is ('C', 'G') and ('G', 'C') the same - so I leave it as is.

In [27]:
names = []
values = []
for item in pairs_dict:
    names.append(item)
    values.append(pairs_dict[item])
    
df = pd.DataFrame()
df['pair'] = names
df['count'] = values
df['pair'] = df['pair'].astype(str)

fig = px.bar(
    df, 
    x='pair', 
    y="count", 
    orientation='v', 
    title='Pair types', 
    height=400, 
    width=800
)
fig.show()

### We can see that the most popular pair is with G and C, the less popular with U and G. And there is only 3 possible combinations of pairs - G and C, U and G, U and A.

## 3. Predicted loop type

In [28]:
sample['predicted_loop_type']

'EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHHHSSSSSSIISSIIIIISSXXXXSSSSSSSHHHHSSSSSSSEEEEEEEEEEEEEEEEEEEEE'

S: paired "Stem" <br>
M: Multiloop  <br>
I: Internal loop <br>
B: Bulge <br>
H: Hairpin loop <br>
E: dangling End <br>
X: eXternal loop <br>

In [29]:
dict(count(sample['predicted_loop_type']))

{'E': 26, 'S': 46, 'H': 15, 'B': 1, 'X': 5, 'I': 14}

In [30]:
loops = []
for j in range(len(train)):
    counts = dict(count(train.iloc[j]['predicted_loop_type']))
    available = ['E', 'S', 'H', 'B', 'X', 'I', 'M']
    row = []
    for item in available:
        try:
            row.append(counts[item] / 107)
        except:
            row.append(0)
    loops.append(row)
    
loops = pd.DataFrame(loops, columns=available)
loops

,E,S,H,B,X,I,M
0,0.242991,0.429907,0.140187,0.009346,0.046729,0.130841,0.000000
1,0.242991,0.616822,0.102804,0.009346,0.000000,0.000000,0.028037
2,0.242991,0.448598,0.140187,0.000000,0.009346,0.112150,0.046729
3,0.252336,0.598131,0.130841,0.018692,0.000000,0.000000,0.000000
4,0.242991,0.654206,0.074766,0.028037,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
2395,0.271028,0.355140,0.214953,0.000000,0.112150,0.046729,0.000000
2396,0.831776,0.130841,0.037383,0.000000,0.000000,0.000000,0.000000
2397,0.242991,0.635514,0.112150,0.000000,0.000000,0.000000,0.009346
2398,0.242991,0.280374,0.112150,0.000000,0.327103,0.037383,0.000000


### Let's check for all samples

In [31]:
res_dict = {}
for j in range(len(train)):
    sam = train.iloc[j]
    prom = dict(count(sam['predicted_loop_type']))
    for item in prom:
        try:
            res_dict[item] += prom[item]
        except:
            res_dict[item] = prom[item]
res_dict

{'E': 85634,
 'S': 113536,
 'H': 28624,
 'B': 3068,
 'X': 7959,
 'I': 12187,
 'M': 5792}

In [32]:
names = []
values = []
for item in res_dict:
    names.append(item)
    values.append(res_dict[item])
    
df = pd.DataFrame()
df['loop_type'] = names
df['count'] = values

In [33]:
fig = px.bar(
    df, 
    x='loop_type', 
    y="count", 
    orientation='v', 
    title='Predicted loop types', 
    height=400, 
    width=600
)
fig.show()

<a id="3"></a>
<h2 style='background:black; border:0; color:white'><center>3. Feature Engineering</center><h2>

## From documentation:

```
At the beginning of the competition, Stanford scientists have data on 3029 RNA sequences of length 107. 
For technical reasons, measurements cannot be carried out on the final bases of these RNA sequences, so we have experimental data (ground truth) in 5 conditions for the first 68 bases.
```

In [34]:
train = pd.concat([train, bases, pairs, loops, pairs_rate], axis=1)
train

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,...,A-U,G-U,E,S,H,B,X,I,M,pairs_rate
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...",...,0.173913,0.000000,0.242991,0.429907,0.140187,0.009346,0.046729,0.130841,0.000000,0.429907
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...",...,0.121212,0.060606,0.242991,0.616822,0.102804,0.009346,0.000000,0.000000,0.028037,0.616822
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...",...,0.125000,0.125000,0.242991,0.448598,0.140187,0.000000,0.009346,0.112150,0.046729,0.448598
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...",...,0.031250,0.062500,0.252336,0.598131,0.130841,0.018692,0.000000,0.000000,0.000000,0.598131
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...",...,0.542857,0.000000,0.242991,0.654206,0.074766,0.028037,0.000000,0.000000,0.000000,0.654206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,2395,id_ff84602f7,GGAAAAUAGCAGAGGAAAUACUAGAGCAAUUGCAAAGGCCGAUCAU...,........((..((......))...)).........(((..........,EEEEEEEESSIISSHHHHHHSSIIISSXXXXXXXXXSSSHHHHHHH...,4.036,1,107,68,"[0.2585, 0.29710000000000003, 0.2748, 0.205000...",...,0.157895,0.105263,0.271028,0.355140,0.214953,0.000000,0.112150,0.046729,0.000000,0.355140
2396,2396,id_ff85fcdba,GGAAAACAAAAACAAACAACAAAAACAAACAACAAAAACAAACAAC...,.................................................,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE...,3.227,1,107,68,"[0.2169, 0.2513, 0.2303, 0.22260000000000002, ...",...,0.285714,0.000000,0.831776,0.130841,0.037383,0.000000,0.000000,0.000000,0.000000,0.130841
2397,2397,id_ffa99f541,GGAAAGCCAUACCUAGGCUUCGGCCUAGGUAUGGCGGUGAUCUGGU...,.....(((((((((((((....)))))))))))))(((((((((((...,EEEEESSSSSSSSSSSSSHHHHSSSSSSSSSSSSSSSSSSSSSSSS...,0.345,0,107,68,"[1.5567, 1.2999, 0.9492, 0.9879, 0.8146, 0.455...",...,0.147059,0.000000,0.242991,0.635514,0.112150,0.000000,0.000000,0.000000,0.009346,0.635514
2398,2398,id_ffe06f3fe,GGAAACGAUAGCAGAAGAGAUCGAUAUAGAGCAUAAGCUAAGAAUA...,.....((((..(....)..))))......(((....)))..........,EEEEESSSSIISHHHHSIISSSSXXXXXXSSSHHHHSSSXXXXXXX...,5.553,0,107,68,"[0.1431, 0.1847, 0.15960000000000002, 0.1466, ...",...,0.200000,0.000000,0.242991,0.280374,0.112150,0.000000,0.327103,0.037383,0.000000,0.280374


In [35]:
train.columns

Index(['index', 'id', 'sequence', 'structure', 'predicted_loop_type',
       'signal_to_noise', 'SN_filter', 'seq_length', 'seq_scored',
       'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10',
       'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10',
       'deg_pH10', 'deg_Mg_50C', 'deg_50C', 'partners', 'A_percent',
       'G_percent', 'C_percent', 'U_percent', 'U-G', 'C-G', 'U-A', 'G-C',
       'A-U', 'G-U', 'E', 'S', 'H', 'B', 'X', 'I', 'M', 'pairs_rate'],
      dtype='object')

In [36]:
train_data = []
for mol_id in train['id'].unique():
    sample_data = train.loc[train['id'] == mol_id]
    for i in range(68): 
        if i < 3:
            previousA = -1
            previousB = -1
            previousC = -1
        else:
            if i%3 == 0:
                previousA = sample_data['sequence'].values[0][i - 3]
                previousB = sample_data['sequence'].values[0][i - 2]
                previousC = sample_data['sequence'].values[0][i - 1]
            if i%3 == 1:
                previousA = sample_data['sequence'].values[0][i - 4]
                previousB = sample_data['sequence'].values[0][i - 3]
                previousC = sample_data['sequence'].values[0][i - 2]
            if i%3 == 2:
                previousA = sample_data['sequence'].values[0][i - 5]
                previousB = sample_data['sequence'].values[0][i - 4]
                previousC = sample_data['sequence'].values[0][i - 3]
            
            
        if i%3 == 0:
            a = sample_data['sequence'].values[0][i]
            b = sample_data['sequence'].values[0][i + 1]
            c = sample_data['sequence'].values[0][i + 2]
            
            nextA = sample_data['sequence'].values[0][i + 3]
            nextB = sample_data['sequence'].values[0][i + 4]
            nextC = sample_data['sequence'].values[0][i + 5]
            next2A = sample_data['sequence'].values[0][i + 6]
            next2B = sample_data['sequence'].values[0][i + 7]
            next2C = sample_data['sequence'].values[0][i + 8]
            next3A = sample_data['sequence'].values[0][i + 9]
            next3B = sample_data['sequence'].values[0][i + 10]
            next3C = sample_data['sequence'].values[0][i + 11]
            
        if i%3 == 1:
            a = sample_data['sequence'].values[0][i - 1]
            b = sample_data['sequence'].values[0][i]
            c = sample_data['sequence'].values[0][i + 1]
            
            nextA = sample_data['sequence'].values[0][i + 2]
            nextB = sample_data['sequence'].values[0][i + 3]
            nextC = sample_data['sequence'].values[0][i + 4]
            next2A = sample_data['sequence'].values[0][i + 5]
            next2B = sample_data['sequence'].values[0][i + 6]
            next2C = sample_data['sequence'].values[0][i + 7]
            next3A = sample_data['sequence'].values[0][i + 8]
            next3B = sample_data['sequence'].values[0][i + 9]
            next3C = sample_data['sequence'].values[0][i + 10]
            
        if i%3 == 2:
            a = sample_data['sequence'].values[0][i - 2]
            b = sample_data['sequence'].values[0][i - 1]
            c = sample_data['sequence'].values[0][i]
            
            nextA = sample_data['sequence'].values[0][i + 1]
            nextB = sample_data['sequence'].values[0][i + 2]
            nextC = sample_data['sequence'].values[0][i + 3]
            next2A = sample_data['sequence'].values[0][i + 4]
            next2B = sample_data['sequence'].values[0][i + 5]
            next2C = sample_data['sequence'].values[0][i + 6]
            next3A = sample_data['sequence'].values[0][i + 7]
            next3B = sample_data['sequence'].values[0][i + 8]
            next3C = sample_data['sequence'].values[0][i + 9]
            
        if a==b and b==c:
            all_the_same = 1
        else:
            all_the_same = 0
            
        if sample_data['structure'].values[0][i] == ')' or sample_data['structure'].values[0][i] == '(':
            isPair = 1
        else:
            isPair = 0
        
        partner_index = sample_data['partners'].values[0][i]
        if partner_index != -1:
            partner =  sample_data['sequence'].values[0][partner_index]
        else:
            partner = -1
        
        sample_tuple = (
            sample_data['id'].values[0], 
            sample_data['sequence'].values[0][i],
            sample_data['structure'].values[0][i], 
            sample_data['predicted_loop_type'].values[0][i],
            sample_data['reactivity'].values[0][i], 
            sample_data['reactivity_error'].values[0][i],
            sample_data['deg_Mg_pH10'].values[0][i], 
            sample_data['deg_error_Mg_pH10'].values[0][i],
            sample_data['deg_pH10'].values[0][i], 
            sample_data['deg_error_pH10'].values[0][i],
            sample_data['deg_Mg_50C'].values[0][i], 
            sample_data['deg_error_Mg_50C'].values[0][i],
            sample_data['deg_50C'].values[0][i], 
            sample_data['deg_error_50C'].values[0][i],
            sample_data['A_percent'].values[0], 
            sample_data['G_percent'].values[0],
            sample_data['C_percent'].values[0], 
            sample_data['U_percent'].values[0],
            sample_data['U-G'].values[0], 
            sample_data['C-G'].values[0],
            sample_data['U-A'].values[0], 
            sample_data['G-C'].values[0],
            sample_data['A-U'].values[0], 
            sample_data['G-U'].values[0], 
            sample_data['E'].values[0],
            sample_data['S'].values[0], 
            sample_data['H'].values[0],
            sample_data['B'].values[0], 
            sample_data['X'].values[0],
            sample_data['I'].values[0], 
            sample_data['M'].values[0],
            sample_data['pairs_rate'].values[0],
            i%3,
            a,
            b,
            c,
            (i%107) / 68,
            all_the_same, 
            isPair,
            previousA,
            previousB,
            previousC,
            nextA,
            nextB,
            nextC,
            next2A,
            next2B,
            next2C,
            next3A,
            next3B,
            next3C,
            partner
        )
        train_data.append(sample_tuple)

In [37]:
train_data = pd.DataFrame(
    train_data, 
    columns=[
        'id', 
        'sequence', 
        'structure', 
        'predicted_loop_type', 
        'reactivity', 
        'reactivity_error', 
        'deg_Mg_pH10', 
        'deg_error_Mg_pH10',
        'deg_pH10', 
        'deg_error_pH10', 
        'deg_Mg_50C', 
        'deg_error_Mg_50C', 
        'deg_50C', 
        'deg_error_50C',
        'A_percent',
        'G_percent',
        'C_percent',
        'U_percent',
        'U-G', 
        'C-G',
        'U-A', 
        'G-C',
        'A-U', 
        'G-U', 
        'E',
        'S', 
        'H',
        'B', 
        'X',
        'I', 
        'M',
        'pairs_rate',
        'codon_position',
        'base_0',
        'base_1',
        'base_2',
        'general_position',
        'all_bases_same',
        'isPair',
        'prevCodon_0',
        'prevCodon_1',
        'prevCodon_2',
        'nextCodon_0',
        'nextCodon_1',
        'nextCodon_2',
        'next2Codon_0',
        'next2Codon_1',
        'next2Codon_2',
        'next3Codon_0',
        'next3Codon_1',
        'next3Codon_2',
        'partner'
    ])
train_data

,id,sequence,structure,predicted_loop_type,reactivity,reactivity_error,deg_Mg_pH10,deg_error_Mg_pH10,deg_pH10,deg_error_pH10,...,nextCodon_0,nextCodon_1,nextCodon_2,next2Codon_0,next2Codon_1,next2Codon_2,next3Codon_0,next3Codon_1,next3Codon_2,partner
0,id_001f94081,G,.,E,0.3297,0.1359,0.7556,0.2613,2.3375,0.2631,...,A,A,A,G,C,U,C,U,A,-1
1,id_001f94081,G,.,E,1.5693,0.2070,2.9830,0.3842,3.5060,0.2860,...,A,A,A,G,C,U,C,U,A,-1
2,id_001f94081,A,.,E,1.1227,0.1633,0.2526,0.1372,0.3008,0.0964,...,A,A,A,G,C,U,C,U,A,-1
3,id_001f94081,A,.,E,0.8686,0.1452,1.3789,0.2531,1.0108,0.1574,...,G,C,U,C,U,A,A,U,A,-1
4,id_001f94081,A,.,E,0.7217,0.1314,0.6376,0.1798,0.2635,0.1000,...,G,C,U,C,U,A,A,U,A,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163195,id_fff546103,C,),S,0.0272,0.0267,0.5059,0.0626,0.4998,0.0996,...,G,C,G,C,C,A,U,G,C,G
163196,id_fff546103,U,),S,0.0382,0.0219,0.2025,0.0411,0.3057,0.0760,...,G,C,G,C,C,A,U,G,C,A
163197,id_fff546103,A,),S,0.0381,0.0218,0.1827,0.0391,0.1642,0.0611,...,G,C,G,C,C,A,U,G,C,U
163198,id_fff546103,G,),S,-0.0066,0.0220,0.3261,0.0509,0.1654,0.0719,...,C,C,A,U,G,C,U,U,C,C


In [38]:
bases = []
for j in range(len(test)):
    counts = dict(count(test.iloc[j]['sequence']))
    bases.append((
        counts['A'] / test.iloc[j]['seq_length'],
        counts['G'] / test.iloc[j]['seq_length'],
        counts['C'] / test.iloc[j]['seq_length'],
        counts['U'] / test.iloc[j]['seq_length']
    ))
    
bases = pd.DataFrame(bases, columns=['A_percent', 'G_percent', 'C_percent', 'U_percent'])
bases

,A_percent,G_percent,C_percent,U_percent
0,0.439252,0.205607,0.186916,0.168224
1,0.384615,0.200000,0.223077,0.192308
2,0.420561,0.271028,0.224299,0.084112
3,0.336449,0.280374,0.233645,0.149533
4,0.207692,0.361538,0.261538,0.169231
...,...,...,...,...
3629,0.346154,0.276923,0.192308,0.184615
3630,0.392308,0.200000,0.200000,0.207692
3631,0.330769,0.230769,0.223077,0.215385
3632,0.392308,0.184615,0.292308,0.130769


In [39]:
pairs = []
all_partners = []
for j in range(len(test)):
    partners = [-1 for i in range(130)]
    pairs_dict = {}
    queue = []
    for i in range(0, len(test.iloc[j]['structure'])):
        if test.iloc[j]['structure'][i] == '(':
            queue.append(i)
        if test.iloc[j]['structure'][i] == ')':
            first = queue.pop()
            try:
                pairs_dict[(test.iloc[j]['sequence'][first], test.iloc[j]['sequence'][i])] += 1
            except:
                pairs_dict[(test.iloc[j]['sequence'][first], test.iloc[j]['sequence'][i])] = 1
                
            partners[first] = i
            partners[i] = first
    
    all_partners.append(partners)
    
    pairs_num = 0
    pairs_unique = [('U', 'G'), ('C', 'G'), ('U', 'A'), ('G', 'C'), ('A', 'U'), ('G', 'U')]
    for item in pairs_dict:
        pairs_num += pairs_dict[item]
    add_tuple = list()
    for item in pairs_unique:
        try:
            add_tuple.append(pairs_dict[item]/pairs_num)
        except:
            add_tuple.append(0)
    pairs.append(add_tuple)
    
pairs = pd.DataFrame(pairs, columns=['U-G', 'C-G', 'U-A', 'G-C', 'A-U', 'G-U'])
pairs

,U-G,C-G,U-A,G-C,A-U,G-U
0,0.066667,0.233333,0.200000,0.233333,0.233333,0.033333
1,0.000000,0.300000,0.166667,0.433333,0.100000,0.000000
2,0.000000,0.363636,0.181818,0.318182,0.136364,0.000000
3,0.000000,0.392857,0.214286,0.178571,0.178571,0.035714
4,0.062500,0.541667,0.104167,0.041667,0.041667,0.208333
...,...,...,...,...,...,...
3629,0.060606,0.181818,0.212121,0.303030,0.121212,0.121212
3630,0.000000,0.434783,0.130435,0.391304,0.043478,0.000000
3631,0.022727,0.272727,0.272727,0.250000,0.181818,0.000000
3632,0.000000,0.277778,0.222222,0.500000,0.000000,0.000000


In [40]:
test['partners'] = all_partners

In [41]:
pairs_rate = []
for j in range(len(test)):
    res = dict(count(test.iloc[j]['structure']))
    pairs_rate.append(res['('] / (test.iloc[j]['seq_length']/2))
    
pairs_rate = pd.DataFrame(pairs_rate, columns=['pairs_rate'])
pairs_rate

,pairs_rate
0,0.560748
1,0.461538
2,0.411215
3,0.523364
4,0.738462
...,...
3629,0.507692
3630,0.353846
3631,0.676923
3632,0.276923


In [42]:
loops = []
for j in range(len(test)):
    counts = dict(count(test.iloc[j]['predicted_loop_type']))
    available = ['E', 'S', 'H', 'B', 'X', 'I', 'M']
    row = []
    for item in available:
        try:
            row.append(counts[item] / test.iloc[j]['seq_length'])
        except:
            row.append(0)
    loops.append(row)
    
loops = pd.DataFrame(loops, columns=available)
loops

,E,S,H,B,X,I,M
0,0.252336,0.560748,0.112150,0.065421,0.009346,0.000000,0.000000
1,0.200000,0.461538,0.107692,0.000000,0.030769,0.161538,0.038462
2,0.299065,0.411215,0.065421,0.000000,0.056075,0.168224,0.000000
3,0.252336,0.523364,0.112150,0.000000,0.000000,0.056075,0.056075
4,0.200000,0.738462,0.061538,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
3629,0.223077,0.507692,0.138462,0.007692,0.000000,0.053846,0.069231
3630,0.215385,0.353846,0.246154,0.000000,0.000000,0.000000,0.184615
3631,0.200000,0.676923,0.092308,0.030769,0.000000,0.000000,0.000000
3632,0.400000,0.276923,0.061538,0.000000,0.007692,0.253846,0.000000


In [43]:
test = pd.concat([test, bases, pairs, loops, pairs_rate], axis=1)
test

,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored,partners,A_percent,G_percent,...,A-U,G-U,E,S,H,B,X,I,M,pairs_rate
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68,"[-1, -1, -1, -1, -1, -1, 66, 65, 64, 60, 59, 5...",0.439252,0.205607,...,0.233333,0.033333,0.252336,0.560748,0.112150,0.065421,0.009346,0.000000,0.000000,0.560748
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91,"[-1, -1, -1, -1, -1, 86, 85, 84, 83, -1, -1, 7...",0.384615,0.200000,...,0.100000,0.000000,0.200000,0.461538,0.107692,0.000000,0.030769,0.161538,0.038462,0.461538
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 6...",0.420561,0.271028,...,0.136364,0.000000,0.299065,0.411215,0.065421,0.000000,0.056075,0.168224,0.000000,0.411215
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68,"[-1, -1, -1, -1, -1, -1, 29, 28, 27, 26, 25, 2...",0.336449,0.280374,...,0.178571,0.035714,0.252336,0.523364,0.112150,0.000000,0.000000,0.056075,0.056075,0.523364
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91,"[-1, -1, -1, -1, -1, 90, 89, 88, 87, 86, 85, 8...",0.207692,0.361538,...,0.041667,0.208333,0.200000,0.738462,0.061538,0.000000,0.000000,0.000000,0.000000,0.738462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3629,3629,id_ff691b7e5,GGAAACUAGCCAUGGGCAGGUUGAAGGUUGGGUGACACUAACUGGA...,........((((((((..((((...((((((......))))))......,EEEEEEEESSSSSSSSMMSSSSIIISSSSSSHHHHHHSSSSSSIII...,130,91,"[-1, -1, -1, -1, -1, -1, -1, -1, 90, 89, 88, 8...",0.346154,0.276923,...,0.121212,0.121212,0.223077,0.507692,0.138462,0.007692,0.000000,0.053846,0.069231,0.507692
3630,3630,id_ff9bf3581,GGAAAUAGCGCCAUAGCCGAUUAUUAUAGGCAAUUUUAGCGAUUUA...,.......(((((...(((..........))).......(((........,EEEEEEESSSSSMMMSSSHHHHHHHHHHSSSMMMMMMMSSSHHHHH...,130,91,"[-1, -1, -1, -1, -1, -1, -1, 62, 61, 60, 59, 5...",0.392308,0.200000,...,0.043478,0.000000,0.215385,0.353846,0.246154,0.000000,0.000000,0.000000,0.184615,0.353846
3631,3631,id_ffc8f96a8,GGAAAGAUGUUCUGAUGAACAUCGGCUGUUCUAGCUUUCAUCUAUC...,.....(((((((....)))))))(((.(((((((((((((((((((...,EEEEESSSSSSSHHHHSSSSSSSSSSBSSSSSSSSSSSSSSSSSSS...,130,91,"[-1, -1, -1, -1, -1, 22, 21, 20, 19, 18, 17, 1...",0.330769,0.230769,...,0.181818,0.000000,0.200000,0.676923,0.092308,0.030769,0.000000,0.000000,0.000000,0.676923
3632,3632,id_ffd7e8cc1,GGAAACCGUUAACCUGCAUCUUCAUGUUAUCGCUUGCGACAGCAAC...,...............................((((((............,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEESSSSSSIIIIIIIII...,130,91,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",0.392308,0.184615,...,0.000000,0.000000,0.400000,0.276923,0.061538,0.000000,0.007692,0.253846,0.000000,0.276923


In [44]:
test_data = []
for mol_id in test['id'].unique():
    sample_data = test.loc[test['id'] == mol_id]
    for i in range(sample_data['seq_scored'].values[0]):
        if i < 3:
            previousA = -1
            previousB = -1
            previousC = -1
        else:
            if i%3 == 0:
                previousA = sample_data['sequence'].values[0][i - 3]
                previousB = sample_data['sequence'].values[0][i - 2]
                previousC = sample_data['sequence'].values[0][i - 1]
            if i%3 == 1:
                previousA = sample_data['sequence'].values[0][i - 4]
                previousB = sample_data['sequence'].values[0][i - 3]
                previousC = sample_data['sequence'].values[0][i - 2]
            if i%3 == 2:
                previousA = sample_data['sequence'].values[0][i - 5]
                previousB = sample_data['sequence'].values[0][i - 4]
                previousC = sample_data['sequence'].values[0][i - 3]
                    
        if i%3 == 0:
            a = sample_data['sequence'].values[0][i]
            b = sample_data['sequence'].values[0][i + 1]
            c = sample_data['sequence'].values[0][i + 2]
            
            nextA = sample_data['sequence'].values[0][i + 3]
            nextB = sample_data['sequence'].values[0][i + 4]
            nextC = sample_data['sequence'].values[0][i + 5]
            next2A = sample_data['sequence'].values[0][i + 6]
            next2B = sample_data['sequence'].values[0][i + 7]
            next2C = sample_data['sequence'].values[0][i + 8]
            next3A = sample_data['sequence'].values[0][i + 9]
            next3B = sample_data['sequence'].values[0][i + 10]
            next3C = sample_data['sequence'].values[0][i + 11]
            
        if i%3 == 1:
            a = sample_data['sequence'].values[0][i - 1]
            b = sample_data['sequence'].values[0][i]
            c = sample_data['sequence'].values[0][i + 1]
            
            nextA = sample_data['sequence'].values[0][i + 2]
            nextB = sample_data['sequence'].values[0][i + 3]
            nextC = sample_data['sequence'].values[0][i + 4]
            next2A = sample_data['sequence'].values[0][i + 5]
            next2B = sample_data['sequence'].values[0][i + 6]
            next2C = sample_data['sequence'].values[0][i + 7]
            next3A = sample_data['sequence'].values[0][i + 8]
            next3B = sample_data['sequence'].values[0][i + 9]
            next3C = sample_data['sequence'].values[0][i + 10]
            
        if i%3 == 2:
            a = sample_data['sequence'].values[0][i - 2]
            b = sample_data['sequence'].values[0][i - 1]
            c = sample_data['sequence'].values[0][i]
            
            nextA = sample_data['sequence'].values[0][i + 1]
            nextB = sample_data['sequence'].values[0][i + 2]
            nextC = sample_data['sequence'].values[0][i + 3]
            next2A = sample_data['sequence'].values[0][i + 4]
            next2B = sample_data['sequence'].values[0][i + 5]
            next2C = sample_data['sequence'].values[0][i + 6]
            next3A = sample_data['sequence'].values[0][i + 7]
            next3B = sample_data['sequence'].values[0][i + 8]
            next3C = sample_data['sequence'].values[0][i + 9]
            
        if a==b and b==c:
            all_the_same = 1
        else:
            all_the_same = 0
            
        if sample_data['structure'].values[0][i] == ')' or sample_data['structure'].values[0][i] == '(':
            isPair = 1
        else:
            isPair = 0
            
        partner_index = sample_data['partners'].values[0][i]
        if partner_index != -1:
            partner =  sample_data['sequence'].values[0][partner_index]
        else:
            partner = -1
            
        sample_tuple = (
            sample_data['id'].values[0] + f'_{i}', 
            sample_data['sequence'].values[0][i],
            sample_data['structure'].values[0][i], 
            sample_data['predicted_loop_type'].values[0][i],
            sample_data['A_percent'].values[0], 
            sample_data['G_percent'].values[0],
            sample_data['C_percent'].values[0], 
            sample_data['U_percent'].values[0],
            sample_data['U-G'].values[0], 
            sample_data['C-G'].values[0],
            sample_data['U-A'].values[0], 
            sample_data['G-C'].values[0],
            sample_data['A-U'].values[0], 
            sample_data['G-U'].values[0], 
            sample_data['E'].values[0],
            sample_data['S'].values[0], 
            sample_data['H'].values[0],
            sample_data['B'].values[0], 
            sample_data['X'].values[0],
            sample_data['I'].values[0], 
            sample_data['M'].values[0],
            sample_data['pairs_rate'].values[0],
            i%3,
            a,
            b,
            c,
            (i%sample_data['seq_scored'].values[0]) / sample_data['seq_scored'].values[0],
            all_the_same, 
            isPair,
            previousA,
            previousB,
            previousC,
            nextA,
            nextB,
            nextC,
            next2A,
            next2B,
            next2C,
            next3A,
            next3B,
            next3C,
            partner
        )
        test_data.append(sample_tuple)

In [45]:
test_data = pd.DataFrame(
    test_data, 
    columns=[
        'id', 
        'sequence', 
        'structure', 
        'predicted_loop_type', 
        'A_percent',
        'G_percent',
        'C_percent',
        'U_percent',
        'U-G', 
        'C-G',
        'U-A', 
        'G-C',
        'A-U', 
        'G-U', 
        'E',
        'S', 
        'H',
        'B', 
        'X',
        'I', 
        'M',
        'pairs_rate',
        'codon_position',
        'base_0',
        'base_1',
        'base_2',
        'general_position',
        'all_bases_same',
        'isPair',
        'prevCodon_0',
        'prevCodon_1',
        'prevCodon_2',        
        'nextCodon_0',
        'nextCodon_1',
        'nextCodon_2',        
        'next2Codon_0',
        'next2Codon_1',
        'next2Codon_2',
        'next3Codon_0',
        'next3Codon_1',
        'next3Codon_2',
        'partner'
    ])
test_data

,id,sequence,structure,predicted_loop_type,A_percent,G_percent,C_percent,U_percent,U-G,C-G,...,nextCodon_0,nextCodon_1,nextCodon_2,next2Codon_0,next2Codon_1,next2Codon_2,next3Codon_0,next3Codon_1,next3Codon_2,partner
0,id_00073f8be_0,G,.,E,0.439252,0.205607,0.186916,0.168224,0.066667,0.233333,...,A,A,A,G,U,A,C,G,A,-1
1,id_00073f8be_1,G,.,E,0.439252,0.205607,0.186916,0.168224,0.066667,0.233333,...,A,A,A,G,U,A,C,G,A,-1
2,id_00073f8be_2,A,.,E,0.439252,0.205607,0.186916,0.168224,0.066667,0.233333,...,A,A,A,G,U,A,C,G,A,-1
3,id_00073f8be_3,A,.,E,0.439252,0.205607,0.186916,0.168224,0.066667,0.233333,...,G,U,A,C,G,A,C,U,U,-1
4,id_00073f8be_4,A,.,E,0.439252,0.205607,0.186916,0.168224,0.066667,0.233333,...,G,U,A,C,G,A,C,U,U,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316222,id_ffda94f24_86,G,),S,0.353846,0.207692,0.207692,0.230769,0.090909,0.272727,...,G,A,U,A,C,A,G,A,G,C
316223,id_ffda94f24_87,G,),S,0.353846,0.207692,0.207692,0.230769,0.090909,0.272727,...,A,C,A,G,A,G,U,G,U,C
316224,id_ffda94f24_88,A,.,X,0.353846,0.207692,0.207692,0.230769,0.090909,0.272727,...,A,C,A,G,A,G,U,G,U,-1
316225,id_ffda94f24_89,U,.,X,0.353846,0.207692,0.207692,0.230769,0.090909,0.272727,...,A,C,A,G,A,G,U,G,U,-1


In [46]:
seq = pd.get_dummies(train_data['sequence'], prefix='Base')
struc = pd.get_dummies(train_data['structure'], prefix='Structure')
loop = pd.get_dummies(train_data['predicted_loop_type'], prefix='Loop')
position = pd.get_dummies(train_data['codon_position'], prefix='Position')
base0 = pd.get_dummies(train_data['base_0'], prefix='Base0')
base1 = pd.get_dummies(train_data['base_1'], prefix='Base1')
base2 = pd.get_dummies(train_data['base_2'], prefix='Base2')
codon0 = pd.get_dummies(train_data['prevCodon_0'], prefix='prevCodon0')
codon1 = pd.get_dummies(train_data['prevCodon_1'], prefix='prevCodon1')
codon2 = pd.get_dummies(train_data['prevCodon_2'], prefix='prevCodon2') 
next_codon0 = pd.get_dummies(train_data['nextCodon_0'], prefix='nextCodon0')
next_codon1 = pd.get_dummies(train_data['nextCodon_1'], prefix='nextCodon1')
next_codon2 = pd.get_dummies(train_data['nextCodon_2'], prefix='nextCodon2')
next2_codon0 = pd.get_dummies(train_data['next2Codon_0'], prefix='next2Codon0')
next2_codon1 = pd.get_dummies(train_data['next2Codon_1'], prefix='next2Codon1')
next2_codon2 = pd.get_dummies(train_data['next2Codon_2'], prefix='next2Codon2')
next3_codon0 = pd.get_dummies(train_data['next3Codon_0'], prefix='next3Codon0')
next3_codon1 = pd.get_dummies(train_data['next3Codon_1'], prefix='next3Codon1')
next3_codon2 = pd.get_dummies(train_data['next3Codon_2'], prefix='next3Codon2')
part = pd.get_dummies(train_data['partner'], prefix='partner')

train_set = pd.concat([seq, struc, loop, position, base0, base1, base2, codon0, codon1, codon2, 
                       next_codon0, next_codon1, next_codon2, next2_codon0, next2_codon1, next2_codon2, next3_codon0, next3_codon1, next3_codon2, part, train_data], 
                      axis=1).drop(['sequence', 'structure', 'predicted_loop_type', 'codon_position', 'base_0', 
                                    'base_1', 'base_2', 'prevCodon_0', 'prevCodon_1', 'prevCodon_2', 
                                    'nextCodon_0', 'nextCodon_1', 'nextCodon_2', 'next2Codon_0', 'next2Codon_1', 'next2Codon_2',
                                    'next3Codon_0', 'next3Codon_1', 'next3Codon_2', 'partner'], axis=1)
train_set

,Base_A,Base_C,Base_G,Base_U,Structure_(,Structure_),Structure_.,Loop_B,Loop_E,Loop_H,...,S,H,B,X,I,M,pairs_rate,general_position,all_bases_same,isPair
0,0,0,1,0,0,0,1,0,1,0,...,0.429907,0.140187,0.009346,0.046729,0.130841,0.000000,0.429907,0.000000,0,0
1,0,0,1,0,0,0,1,0,1,0,...,0.429907,0.140187,0.009346,0.046729,0.130841,0.000000,0.429907,0.014706,0,0
2,1,0,0,0,0,0,1,0,1,0,...,0.429907,0.140187,0.009346,0.046729,0.130841,0.000000,0.429907,0.029412,0,0
3,1,0,0,0,0,0,1,0,1,0,...,0.429907,0.140187,0.009346,0.046729,0.130841,0.000000,0.429907,0.044118,1,0
4,1,0,0,0,0,0,1,0,1,0,...,0.429907,0.140187,0.009346,0.046729,0.130841,0.000000,0.429907,0.058824,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163195,0,1,0,0,0,1,0,0,0,0,...,0.616822,0.093458,0.028037,0.000000,0.000000,0.018692,0.616822,0.926471,0,1
163196,0,0,0,1,0,1,0,0,0,0,...,0.616822,0.093458,0.028037,0.000000,0.000000,0.018692,0.616822,0.941176,0,1
163197,1,0,0,0,0,1,0,0,0,0,...,0.616822,0.093458,0.028037,0.000000,0.000000,0.018692,0.616822,0.955882,0,1
163198,0,0,1,0,0,1,0,0,0,0,...,0.616822,0.093458,0.028037,0.000000,0.000000,0.018692,0.616822,0.970588,0,1


In [47]:
seq = pd.get_dummies(test_data['sequence'], prefix='Base')
struc = pd.get_dummies(test_data['structure'], prefix='Structure')
loop = pd.get_dummies(test_data['predicted_loop_type'], prefix='Loop')
position = pd.get_dummies(test_data['codon_position'], prefix='Position')
base0 = pd.get_dummies(test_data['base_0'], prefix='Base0')
base1 = pd.get_dummies(test_data['base_1'], prefix='Base1')
base2 = pd.get_dummies(test_data['base_2'], prefix='Base2')
codon0 = pd.get_dummies(test_data['prevCodon_0'], prefix='prevCodon0')
codon1 = pd.get_dummies(test_data['prevCodon_1'], prefix='prevCodon1')
codon2 = pd.get_dummies(test_data['prevCodon_2'], prefix='prevCodon2') 
next_codon0 = pd.get_dummies(test_data['nextCodon_0'], prefix='nextCodon0')
next_codon1 = pd.get_dummies(test_data['nextCodon_1'], prefix='nextCodon1')
next_codon2 = pd.get_dummies(test_data['nextCodon_2'], prefix='nextCodon2') 
next2_codon0 = pd.get_dummies(test_data['next2Codon_0'], prefix='next2Codon0')
next2_codon1 = pd.get_dummies(test_data['next2Codon_1'], prefix='next2Codon1')
next2_codon2 = pd.get_dummies(test_data['next2Codon_2'], prefix='next2Codon2')
next3_codon0 = pd.get_dummies(test_data['next3Codon_0'], prefix='next3Codon0')
next3_codon1 = pd.get_dummies(test_data['next3Codon_1'], prefix='next3Codon1')
next3_codon2 = pd.get_dummies(test_data['next3Codon_2'], prefix='next3Codon2')
part = pd.get_dummies(test_data['partner'], prefix='partner')

test_set = pd.concat([seq, struc, loop, position, base0, base1, base2, codon0, codon1, codon2, 
                       next_codon0, next_codon1, next_codon2, next2_codon0, next2_codon1, next2_codon2, next3_codon0, next3_codon1, next3_codon2, part, test_data], 
                      axis=1).drop(['sequence', 'structure', 'predicted_loop_type', 'codon_position', 'base_0', 
                                    'base_1', 'base_2', 'prevCodon_0', 'prevCodon_1', 'prevCodon_2', 
                                    'nextCodon_0', 'nextCodon_1', 'nextCodon_2', 'next2Codon_0', 'next2Codon_1', 'next2Codon_2',
                                    'next3Codon_0', 'next3Codon_1', 'next3Codon_2', 'partner'], axis=1)
test_set

,Base_A,Base_C,Base_G,Base_U,Structure_(,Structure_),Structure_.,Loop_B,Loop_E,Loop_H,...,S,H,B,X,I,M,pairs_rate,general_position,all_bases_same,isPair
0,0,0,1,0,0,0,1,0,1,0,...,0.560748,0.112150,0.065421,0.009346,0.000000,0.0,0.560748,0.000000,0,0
1,0,0,1,0,0,0,1,0,1,0,...,0.560748,0.112150,0.065421,0.009346,0.000000,0.0,0.560748,0.014706,0,0
2,1,0,0,0,0,0,1,0,1,0,...,0.560748,0.112150,0.065421,0.009346,0.000000,0.0,0.560748,0.029412,0,0
3,1,0,0,0,0,0,1,0,1,0,...,0.560748,0.112150,0.065421,0.009346,0.000000,0.0,0.560748,0.044118,1,0
4,1,0,0,0,0,0,1,0,1,0,...,0.560748,0.112150,0.065421,0.009346,0.000000,0.0,0.560748,0.058824,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316222,0,0,1,0,0,1,0,0,0,0,...,0.507692,0.107692,0.007692,0.023077,0.138462,0.0,0.507692,0.945055,0,1
316223,0,0,1,0,0,1,0,0,0,0,...,0.507692,0.107692,0.007692,0.023077,0.138462,0.0,0.507692,0.956044,0,1
316224,1,0,0,0,0,0,1,0,0,0,...,0.507692,0.107692,0.007692,0.023077,0.138462,0.0,0.507692,0.967033,0,0
316225,0,0,0,1,0,0,1,0,0,0,...,0.507692,0.107692,0.007692,0.023077,0.138462,0.0,0.507692,0.978022,0,0


In [48]:
train_target = train_set[['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']]
train_set = train_set.drop(['id', 'reactivity', 'reactivity_error', 'deg_Mg_pH10', 'deg_error_Mg_pH10', 'deg_pH10', 'deg_error_pH10',
                            'deg_Mg_50C', 'deg_error_Mg_50C', 'deg_50C', 'deg_error_50C'], axis=1)

In [49]:
test_id = test_set['id']
test_set = test_set.drop(['id'], axis=1)

In [50]:
test_set

,Base_A,Base_C,Base_G,Base_U,Structure_(,Structure_),Structure_.,Loop_B,Loop_E,Loop_H,...,S,H,B,X,I,M,pairs_rate,general_position,all_bases_same,isPair
0,0,0,1,0,0,0,1,0,1,0,...,0.560748,0.112150,0.065421,0.009346,0.000000,0.0,0.560748,0.000000,0,0
1,0,0,1,0,0,0,1,0,1,0,...,0.560748,0.112150,0.065421,0.009346,0.000000,0.0,0.560748,0.014706,0,0
2,1,0,0,0,0,0,1,0,1,0,...,0.560748,0.112150,0.065421,0.009346,0.000000,0.0,0.560748,0.029412,0,0
3,1,0,0,0,0,0,1,0,1,0,...,0.560748,0.112150,0.065421,0.009346,0.000000,0.0,0.560748,0.044118,1,0
4,1,0,0,0,0,0,1,0,1,0,...,0.560748,0.112150,0.065421,0.009346,0.000000,0.0,0.560748,0.058824,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316222,0,0,1,0,0,1,0,0,0,0,...,0.507692,0.107692,0.007692,0.023077,0.138462,0.0,0.507692,0.945055,0,1
316223,0,0,1,0,0,1,0,0,0,0,...,0.507692,0.107692,0.007692,0.023077,0.138462,0.0,0.507692,0.956044,0,1
316224,1,0,0,0,0,0,1,0,0,0,...,0.507692,0.107692,0.007692,0.023077,0.138462,0.0,0.507692,0.967033,0,0
316225,0,0,0,1,0,0,1,0,0,0,...,0.507692,0.107692,0.007692,0.023077,0.138462,0.0,0.507692,0.978022,0,0


In [51]:
drop_columns = ['partner_-1', 'prevCodon1_-1', 'prevCodon2_-1', 'isPair', 'pairs_rate']

train_set = train_set.drop(drop_columns, axis=1)
test_set = test_set.drop(drop_columns, axis=1)

<a id="4"></a>
<h2 style='background:black; border:0; color:white'><center>4. Keras Neural Network Model</center><h2>

In [52]:
def MCRMSE(y_true, y_pred):
    colwise_mse = K.mean(K.square(y_true - y_pred))
    return K.mean(K.sqrt(colwise_mse))

In [53]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(101),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(500, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(3, activation="elu")
    ])
    model.compile(optimizer='adam', loss=MCRMSE)
    return model

In [54]:
from sklearn.metrics import mean_squared_error as mse
import math

def rmse(y_true, y_pred):
    return math.sqrt(mse(y_true, y_pred)) / 3

In [55]:
train_target.columns

Index(['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C'], dtype='object')

In [56]:
target = train_target[['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']]

In [57]:
preds_df = pd.DataFrame()
preds_df['id'] = test_id
preds_df.loc[:, ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']] = 0
res = target.copy()
for n, (tr, te) in enumerate(KFold(n_splits=10, random_state=666, shuffle=True).split(target)):
    print(f'Fold {n}')
    
    model = create_model()
    
    model.fit(
        train_set.values[tr],
        target.values[tr],
        epochs=45, 
        batch_size=64
    )
    
    preds_df.loc[:, ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']] += model.predict(test_set)
    res.loc[te, ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']] = model.predict(train_set.values[te])
    
preds_df.loc[:, ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']] /= (n+1)

Fold 0
Epoch 1/45
2295/2295 [==============================] - 8s 4ms/step - loss: 0.6201
Epoch 2/45
2295/2295 [==============================] - 9s 4ms/step - loss: 0.5524
Epoch 3/45
2295/2295 [==============================] - 9s 4ms/step - loss: 0.5401
Epoch 4/45
2295/2295 [==============================] - 8s 4ms/step - loss: 0.5377
Epoch 5/45
2295/2295 [==============================] - 9s 4ms/step - loss: 0.5345
Epoch 6/45
2295/2295 [==============================] - 8s 4ms/step - loss: 0.5311
Epoch 7/45
2295/2295 [==============================] - 9s 4ms/step - loss: 0.5301
Epoch 8/45
2295/2295 [==============================] - 8s 4ms/step - loss: 0.5276
Epoch 9/45
2295/2295 [==============================] - 8s 4ms/step - loss: 0.5244
Epoch 10/45
2295/2295 [==============================] - 8s 4ms/step - loss: 0.5273
Epoch 11/45
2295/2295 [==============================] - 9s 4ms/step - loss: 0.5242
Epoch 12/45
2295/2295 [==============================] - 9s 4ms/step - loss: 0

In [58]:
metrics = []
for _target in target.columns:
    metrics.append(rmse(target.loc[:, _target], res.loc[:, _target]))

### Let's check our cross validation score

In [59]:
print(f'OOF Metric: {np.mean(metrics)}')

OOF Metric: 0.21704689245770345


<a id="5"></a>
<h2 style='background:black; border:0; color:white'><center>5. Prepare submission file</center><h2>

### So the final step is to prepare submission file

In [60]:
preds_df

,id,reactivity,deg_Mg_pH10,deg_Mg_50C
0,id_00073f8be_0,0.668900,0.684415,0.570678
1,id_00073f8be_1,2.175314,3.083722,3.012901
2,id_00073f8be_2,1.159188,0.604649,0.751597
3,id_00073f8be_3,1.324686,1.050436,1.319201
4,id_00073f8be_4,0.796161,0.616172,0.751411
...,...,...,...,...
316222,id_ffda94f24_86,0.095126,0.196408,0.161960
316223,id_ffda94f24_87,0.257761,0.421599,0.329831
316224,id_ffda94f24_88,0.672102,0.710604,0.682703
316225,id_ffda94f24_89,0.617345,0.691335,0.592160


In [61]:
sub = pd.merge(sub[['id_seqpos']], preds_df, left_on='id_seqpos', right_on='id', how='left').drop(['id'],axis=1)
sub = sub.fillna(0)
sub['deg_pH10'] = 0
sub['deg_50C'] = 0
sub = sub[['id_seqpos', 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']]
sub

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.668900,0.684415,0,0.570678,0
1,id_00073f8be_1,2.175314,3.083722,0,3.012901,0
2,id_00073f8be_2,1.159188,0.604649,0,0.751597,0
3,id_00073f8be_3,1.324686,1.050436,0,1.319201,0
4,id_00073f8be_4,0.796161,0.616172,0,0.751411,0
...,...,...,...,...,...,...
457948,id_ffda94f24_125,0.000000,0.000000,0,0.000000,0
457949,id_ffda94f24_126,0.000000,0.000000,0,0.000000,0
457950,id_ffda94f24_127,0.000000,0.000000,0,0.000000,0
457951,id_ffda94f24_128,0.000000,0.000000,0,0.000000,0


In [62]:
sub.to_csv('submission.csv', index=False)